In [8]:
!pip install requests
!pip install pandas

In [9]:
from bs4 import BeautifulSoup 
import pandas as pd
import requests 
import re

pd.set_option ('display.max_columns', None)

In [18]:
url = "https://moviesdatabase.p.rapidapi.com/titles"
querystring = {"genre": "Drama", "startYear": "2010", "endYear": "2024"}
headers = {
        "X-RapidAPI-Key": "431e779057msh1f0c87c2937a329p125efcjsn45064e4b4925",
        "X-RapidAPI-Host": "moviesdatabase.p.rapidapi.com"
    }
response = requests.get(url, headers=headers, params=querystring)


    

In [19]:
datos_sucios = response.json()

In [20]:
datos_sucios ['next']

'/titles?endYear=2024&startYear=2010&genre=Drama&page=2'

In [ ]:
datos_medios = datos_sucios ['results'][6]
df_medios = pd.DataFrame (datos_medios)
df_medios.head ()

In [ ]:
len (datos_sucios ['results'])

10

In [ ]:
for diccionario  in datos_sucios ['results']:
    clave_primaryImage = diccionario.get ('primaryImage')
    print (clave_primaryImage)
    

{'id': 'rm479879936', 'width': 1200, 'height': 1800, 'url': 'https://m.media-amazon.com/images/M/MV5BZTIwZmViMWItMDA5Ny00OTczLTg2NzYtYzEwM2ZjOTI3NmVlXkEyXkFqcGdeQXVyMzY0MTE3NzU@._V1_.jpg', 'caption': {'plainText': 'Vampire: The Masquerade - Bloodlines 2 (2024)', '__typename': 'Markdown'}, '__typename': 'Image'}
{'id': 'rm2747813633', 'width': 456, 'height': 612, 'url': 'https://m.media-amazon.com/images/M/MV5BMjQ0ZjgxZTUtMzNkNy00NmIxLWFlYzgtOTkzNDUyYzUzNjJlXkEyXkFqcGdeQXVyMTY4MjUxMzgz._V1_.jpg', 'caption': {'plainText': 'La Storia (2024)', '__typename': 'Markdown'}, '__typename': 'Image'}
{'id': 'rm231164417', 'width': 650, 'height': 1000, 'url': 'https://m.media-amazon.com/images/M/MV5BMDg0YTJmMTAtOWEzYi00NDk1LTg5MmEtZWMwNjgxMGFjNzkyXkEyXkFqcGdeQXVyMTUwMjg2Nzg0._V1_.jpg', 'caption': {'plainText': 'Boy Swallows Universe (2024)', '__typename': 'Markdown'}, '__typename': 'Image'}
None
{'id': 'rm4127568385', 'width': 792, 'height': 1080, 'url': 'https://m.media-amazon.com/images/M/MV5BZWI

In [21]:
def limpiar_pelis(datos):
    lista_pelis = []
    if 'results' in datos:
        for peli in datos['results']:
            tipo_peli = "Película"
            nombre_peli = peli.get('titleText', {}).get('text', 'Unknown')
            id_peli = peli.get('id', 'Unknown')
            lanza_peli = peli.get('releaseDate', {})

            año_peli = lanza_peli.get('year', 'Unknown')
            mes_peli = lanza_peli.get('month', 'Unknown')
            if mes_peli is None:
                mes_peli = 'Unknown'
            elif isinstance(mes_peli, int):
                mes_peli = int(mes_peli)

            movie_info = (tipo_peli, nombre_peli, año_peli, mes_peli, id_peli)
            lista_pelis.append(movie_info)
    return lista_pelis


datos_peliculas = llamar_pelis()
pelis_procesadas = limpiar_pelis(datos_peliculas)

for peli in pelis_procesadas:
    print(peli)

('Película', 'Vampire: The Masquerade - Bloodlines 2', 2024, 'Unknown', 'tt10064424')
('Película', 'La Storia', 2024, 1, 'tt10345822')
('Película', 'Boy Swallows Universe', 2024, 1, 'tt10399902')
('Película', 'Dune: The Prophecy', 2024, 'Unknown', 'tt10466872')
('Película', 'The Divine Comedy: Inferno, Purgatory and Paradise', 2024, 9, 'tt10521010')
('Película', 'Those About to Die', 2024, 7, 'tt10569934')
('Película', "Mom's Coming", 2024, 11, 'tt10579986')
('Película', 'Love Song and Power', 2024, 1, 'tt10597228')
('Película', "Dark Forest: Dante's Sins. Virgil appears.", 2024, 9, 'tt10616590')
('Película', "Beatrice Descends into Limbo. Virgil's Mission", 2024, 9, 'tt10617026')
